In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!sudo apt-get update -y && sudo apt-get install -y aria2


In [ ]:
#!aria2c -x 16 -s 16 -k 16M -d "/content/drive/MyDrive/Colab_Notebooks/dataset" -o "physionet.zip" "PresignedUrl"


In [ ]:
!sudo apt-get update -y
!sudo apt-get install -y p7zip-full
!7z x "/content/drive/MyDrive/Colab_Notebooks/dataset/physionet.zip" -o"/content/data"


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,227 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packa

In [ ]:
from src.mask_generation import process_all_parallel
from src.detect_bad_masks import get_bad_masks_parallel,regenerate_missing


train_path="/content/data/train"

process_all_parallel(train_path,workers=12,print_overlay=False, thickness=4)
get_bad_masks_parallel(train_path,delete=True,workers=12)
regenerate_missing(train_path)

Processing ECGs: 100%|██████████| 977/977 [07:40<00:00,  2.12ecg/s]

DONE | ok=977 bad=0 total=977



Scanning npz files: 100%|██████████| 977/977 [00:13<00:00, 73.12it/s]


Total npz: 977
Bad npz: 0
The corrupted masks will be deleted? True
Missing masks: 0


Regenerating: 0it [00:00, ?it/s]


In [ ]:
!pip -q install timm==0.9.2 segmentation-models-pytorch==0.3.3 unzip kaggle


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 12.0 MB/s eta 0:00:00


In [ ]:
import os, glob, random
import numpy as np
from PIL import Image
import cv2
import os, glob

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import segmentation_models_pytorch as smp

SEED = 42
random.seed(SEED);
np.random.seed(SEED);
torch.manual_seed(SEED)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

DATA_ROOT = "/content/data/train"
H_T = 864
W_T = 1120 # half-res of (1700,2200)
K = 13
allowed_types = None
use_warp = True
dataset_cache_dir = f"{DATA_ROOT}_cache_ecg"
BATCH_SIZE = 24
NUM_WORKERS = 8
weight_decay=1e-4
EPOCHS = 20

In [ ]:
from torch.utils.data.dataloader import default_collate

def collate_skip_none(batch):
    batch = [b for b in batch if b is not None]
    if len(batch) == 0:
        return None
    return default_collate(batch)


In [ ]:
import os, glob
from src.ECGSegDataset import ECGSegDataset

folders = sorted(
    [p for p in glob.glob(os.path.join(DATA_ROOT, "*")) if os.path.isdir(p) and os.path.basename(p).isdigit()])
random.shuffle(folders)

val_n = max(200, int(0.1 * len(folders)))
val_n = min(val_n, len(folders) - 1)
val_f = folders[:val_n]
tr_f = folders[val_n:]

dataset_cache_dir = "/content/cache_ecgseg"   # keep cache local for speed

train_ds = ECGSegDataset(
    tr_f, H_T, W_T, K,
    use_warp=use_warp,
    cache_dir=dataset_cache_dir,
    skip_errors=True,
    error_log_path="/content/bad_samples.txt",
    return_paths=True,
)

val_clean_ds = ECGSegDataset(
    val_f, H_T, W_T, K,
    allowed_types={"0001"},
    use_warp=True,
    cache_dir=dataset_cache_dir,
    skip_errors=True,
    error_log_path="/content/bad_samples.txt",
    return_paths=True,
)

val_hard_ds = ECGSegDataset(
    val_f, H_T, W_T, K,
    allowed_types={"0005","0006","0009"},
    use_warp=True,
    cache_dir=dataset_cache_dir,
    skip_errors=True,
    error_log_path="/content/bad_samples.txt",
    return_paths=True,
)

dl_kwargs = dict(
    num_workers=NUM_WORKERS,
    collate_fn=collate_skip_none,
    pin_memory=True,
    persistent_workers=False,
)
if NUM_WORKERS > 0:
    dl_kwargs["prefetch_factor"] = 2

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, **dl_kwargs)
val_loader_clean = DataLoader(val_clean_ds, batch_size=1, shuffle=False, **dl_kwargs)
val_loader_hard  = DataLoader(val_hard_ds,  batch_size=1, shuffle=False, **dl_kwargs)

len(tr_f), len(val_f)


(777, 200)

In [ ]:
def get_model(model_path):
    K = 13  # number of channels

    model = smp.Unet(
        encoder_name="resnet34",
        encoder_weights=None,  # IMPORTANT: None for inference
        in_channels=1,
        classes=K,
    ).to(DEVICE)
    ckpt = torch.load(
        model_path,
        map_location=DEVICE
    )
    model.load_state_dict(ckpt["model"])
    return model
model_path="/content/models/best_unet_resnet34_halfres_new.pt"
model = get_model(model_path)

In [ ]:
def set_encoder_trainable(model, trainable: bool):
    for p in model.encoder.parameters():
        p.requires_grad = trainable

def freeze_bn_stats(m):
    if isinstance(m, nn.BatchNorm2d):
        m.eval()

def dice_loss(logits, y, eps=1e-6):
    p = torch.sigmoid(logits)
    num = 2*(p*y).sum((2,3))
    den = (p+y).sum((2,3)) + eps
    return (1 - num/den).mean()

def loss_fn(logits, y):
    return bce(logits, y) + dice_loss(logits, y)


In [ ]:
bce = nn.BCEWithLogitsLoss()

model.train()  # full model train mode
model.apply(freeze_bn_stats)


encoder_params = list(model.encoder.parameters())
other_params = [p for n,p in model.named_parameters() if not n.startswith("encoder.")]

opt = optim.AdamW(
    [
        {"params": encoder_params, "lr": 1e-6},   # smaller LR
        {"params": other_params,   "lr": 5e-6},   # larger LR
    ],
    weight_decay=1e-4
  )
scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))


/tmp/ipython-input-600744964.py:17: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))


In [ ]:
from tqdm.auto import tqdm

def run_epoch(loader, train=True, log_every=50):
    model.train(train)
    if train:
      model.apply(freeze_bn_stats)

    tot, n = 0.0, 0

    pbar = tqdm(loader, desc="train" if train else "val", leave=False)
    for step, batch in enumerate(pbar, start=1):
        if batch is None:
            continue
        x, y, _ = batch

        x = x.to(DEVICE, non_blocking=True)
        y = y.to(DEVICE, non_blocking=True)

        with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
            logits = model(x)
            loss = loss_fn(logits, y)

        if train:
            opt.zero_grad(set_to_none=True)
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()

        tot += loss.item()
        n += 1
        avg = tot / n

        # Update progress bar postfix every step (lightweight)
        pbar.set_postfix(loss=f"{loss.item():.4f}", avg=f"{avg:.4f}")

        # Optional: also print a line every log_every steps
        if log_every and (step % log_every == 0):
            tqdm.write(f"[{'train' if train else 'val'}] step {step}/{len(loader)}  loss={loss.item():.4f}  avg={avg:.4f}")

    return tot / max(n, 1)




In [ ]:
from torch.utils.data import DataLoader

debug_loader = DataLoader(
    val_loader_hard.dataset,   # reuse the same dataset
    batch_size=1,
    shuffle=False,
    num_workers=0,         # IMPORTANT: avoid worker/process issues
    pin_memory=False
)

debug_x, debug_y,png_path = next(iter(debug_loader))
debug_x = debug_x.contiguous()
debug_y = debug_y.contiguous()
print(f"debug_x shape: {debug_x.shape}")
print(f"debug_y shape: {debug_y.shape}")
print(f"debug_png_path: {png_path}")


debug_x shape: torch.Size([1, 1, 864, 1120])
debug_y shape: torch.Size([1, 13, 864, 1120])
debug_png_path: (np.str_('/content/data/train/1228736690/1228736690-0009.png'),)


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch

def save_pred_debug_epoch(
    model,
    sample_x, sample_y,
    epoch,
    out_dir="debug_train_evolution",
    ch_names=None,              # list of channel names length K (optional)
    ch_idxs=(1, 12),            # e.g. (Lead II index, Rhythm index)
    thr=0.3,
    device="cuda"
):
    """
    Saves a PNG showing:
      - input image
      - GT mask for selected channels
      - Pred mask for selected channels
    Assumes x shape: [1,1,H,W], y shape: [1,K,H,W]
    """
    os.makedirs(out_dir, exist_ok=True)

    model.eval()
    with torch.no_grad():
        x = sample_x.to(device)
        y = sample_y.to("cpu")

        logits = model(x).detach().to("cpu")
        probs = torch.sigmoid(logits)

    # Use first item in batch
    img = x[0, 0].detach().to("cpu").numpy()  # [H,W]

    # Build figure (1 row per channel, 3 columns)
    rows = len(ch_idxs)
    fig, axs = plt.subplots(rows, 3, figsize=(12, 4 * rows))

    if rows == 1:
        axs = np.expand_dims(axs, axis=0)

    for r, c in enumerate(ch_idxs):
        gt = y[0, c].numpy()
        pr = probs[0, c].numpy()

        name = f"ch{c}"
        if ch_names is not None and c < len(ch_names):
            name = ch_names[c]

        axs[r, 0].imshow(img, cmap="gray")
        axs[r, 0].set_title("Input")
        axs[r, 0].axis("off")

        axs[r, 1].imshow(gt, cmap="gray")
        axs[r, 1].set_title(f"GT: {name}")
        axs[r, 1].axis("off")

        axs[r, 2].imshow(pr > thr, cmap="gray")
        axs[r, 2].set_title(f"Pred (thr={thr}): {name}")
        axs[r, 2].axis("off")

    plt.tight_layout()
    path = os.path.join(out_dir, f"epoch_{epoch:03d}.png")
    plt.savefig(path, dpi=150)
    plt.close(fig)

    return path


In [ ]:
DEBUG_DIR = "debug_train_evolution"

best_val = float("inf")
for ep in range(1, EPOCHS+1):
    print(f"\n=== Epoch {ep}/{EPOCHS} ===")
    tr = run_epoch(train_loader, train=True, log_every=50)
    va_clean = run_epoch(val_loader_clean, train=False, log_every=0)
    va_hard  = run_epoch(val_loader_hard,  train=False, log_every=0)

    print(f"epoch {ep}/{EPOCHS} | train {tr:.4f} | val_clean {va_clean:.4f} | val_hard {va_hard:.4f}")


    # ✅ Save debug prediction snapshot each epoch (cheap)
    snap = save_pred_debug_epoch(
        model,
        debug_x, debug_y,
        epoch=ep,
        out_dir=DEBUG_DIR,
        ch_idxs=(1, 12),
        thr=0.3,
        device=DEVICE
    )
    print("saved debug snapshot:", snap)

    if va_hard < best_val:
        best_val = va_hard
        torch.save({"model": model.state_dict(), "val_loss": va_hard},
                   "best_unet_resnet34_halfres.pt")
        print("saved best checkpoint")



=== Epoch 1/20 ===


train:   0%|          | 0/33 [00:00<?, ?it/s]

/tmp/ipython-input-3860769458.py:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):


val:   0%|          | 0/200 [00:00<?, ?it/s]

val:   0%|          | 0/200 [00:00<?, ?it/s]

...

epoch 9/20 | train 0.5875 | val_clean 0.2441 | val_hard 0.7497


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x792fd4bca2a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1618, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.12/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/connection.py", line 1136, in wait
    ready = selector.select(timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
    

saved debug snapshot: debug_train_evolution/epoch_009.png

=== Epoch 10/20 ===


train:   0%|          | 0/33 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import os, glob
import numpy as np
from PIL import Image
import cv2
import torch

def save_pred_overlays(val_f, out_dir="/content/overlays", n=50, thr=0.5):
    os.makedirs(out_dir, exist_ok=True)
    model.eval()

    n = min(n, len(val_f))
    for i in range(n):
        d = val_f[i]
        ecg_id = os.path.basename(d)

        png_path = sorted(glob.glob(os.path.join(d, "*0001.png")))[0]
        npz_path = sorted(glob.glob(os.path.join(d, "mask-*.npz")))[0]

        img0 = np.array(Image.open(png_path).convert("L"), dtype=np.uint8)
        img = cv2.resize(img0.astype(np.float32)/255.0, (W_T, H_T), interpolation=cv2.INTER_AREA)
        x = torch.from_numpy(img[None,None,...]).float().to(DEVICE)

        z = np.load(npz_path, allow_pickle=True)
        gt0 = z["masks"]
        gt = np.zeros((H_T, W_T, K), dtype=np.uint8)
        for k in range(K):
            gt[..., k] = cv2.resize(gt0[..., k], (W_T, H_T), interpolation=cv2.INTER_NEAREST)

        with torch.no_grad():
            logits = model(x)[0].detach().cpu().numpy()  # (K,H,W)
            prob = 1/(1+np.exp(-logits))

        pred_union = (prob.max(axis=0) > thr).astype(np.uint8)
        gt_union   = (gt.max(axis=2) > 0).astype(np.uint8)

        base = (img*255).astype(np.uint8)
        base_rgb = np.stack([base]*3, axis=-1)

        # Pred overlay in red
        out_pred = base_rgb.copy()
        m = pred_union.astype(bool)
        out_pred[m] = (0.5*out_pred[m] + 0.5*np.array([255,0,0])).astype(np.uint8)

        # GT overlay in green (optional but very useful)
        out_gt = base_rgb.copy()
        g = gt_union.astype(bool)
        out_gt[g] = (0.5*out_gt[g] + 0.5*np.array([0,255,0])).astype(np.uint8)

        Image.fromarray(out_pred).save(os.path.join(out_dir, f"{i:04d}_{ecg_id}_pred.png"))
        Image.fromarray(out_gt).save(os.path.join(out_dir, f"{i:04d}_{ecg_id}_gt.png"))

    return out_dir

out_dir = save_pred_overlays(val_f, out_dir="/content/overlays", n=1, thr=0.3)
print("Saved to:", out_dir)
